In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

In [2]:
path = '../Entrenamientos/Procesado_Tramos.xlsx'
df = pd.read_excel(path)

In [3]:
#df = df[df["viento_aparente"]!=-np.inf]
df = df[df["porc"]!=np.inf]
df["puntos_en_tramo"]= df["tramo_final"]-df["tramo_inicial"]
df = df[df["puntos_en_tramo"]>250]
df = df[df["porc"]>2]
df = df[df["prueba"]=="20180406_LEVS_195_90_R_E.gpx"]

In [4]:
df.describe()

,tramo,tramo_inicial,tramo_final,pwr,time_seg,dist,spd,viento_aparente,porc,dist_acum,puntos_en_tramo
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,21.400000,2333.600000,2907.400000,235.285442,1830.800000,7072.637448,14.604539,5.328055,5.372944,39510.098295,573.800000
std,13.221195,1833.527965,1877.566856,16.326075,800.207286,2582.611236,4.209147,1.721780,1.633355,25576.666498,218.538784
min,1.000000,65.000000,432.000000,215.401244,980.000000,3795.770186,11.200633,2.516139,3.207283,5773.000000,312.000000
25%,19.000000,1305.000000,2003.000000,222.291330,1220.000000,4819.000000,11.537360,5.388943,4.181100,27641.000000,367.000000
50%,21.000000,2017.000000,2813.000000,239.264316,1618.000000,8252.417053,12.132561,5.592770,5.870399,37196.000000,696.000000
75%,31.000000,3587.000000,3899.000000,243.882193,2575.000000,9191.000000,17.702449,5.944084,6.468743,53876.919962,698.000000
max,35.000000,4694.000000,5390.000000,255.588127,2761.000000,9305.000000,20.449691,7.198339,7.137195,73063.571515,796.000000


In [5]:
df.head()

,tramo,tramo_inicial,tramo_final,pwr,time_seg,dist,spd,viento_aparente,porc,dist_acum,prueba,puntos_en_tramo
1,1,65,432,255.588127,980,4819.000000,17.702449,7.198339,4.181100,5773.000000,20180406_LEVS_195_90_R_E.gpx,367
19,19,1305,2003,239.264316,1618,9191.000000,20.449691,5.944084,3.207283,27641.000000,20180406_LEVS_195_90_R_E.gpx,698
21,21,2017,2813,243.882193,2761,9305.000000,12.132561,2.516139,7.137195,37196.000000,20180406_LEVS_195_90_R_E.gpx,796
31,31,3587,3899,222.291330,1220,3795.770186,11.200633,5.592770,5.870399,53876.919962,20180406_LEVS_195_90_R_E.gpx,312
35,35,4694,5390,215.401244,2575,8252.417053,11.537360,5.388943,6.468743,73063.571515,20180406_LEVS_195_90_R_E.gpx,696


In [55]:
def simulador2(lista_tramos, nivel, registro_func, potencia_entrada, potencia_media_func, tiempo_acum_func, potencia_acum_func):
    #print(nivel, potencia_entrada, potencia_media_func, tiempo_acum_func, potencia_acum_func)
    #input("Press Enter to continue...")
    
    global contador_resultados        
    global model        
     
    if len(lista_tramos)==0:
        paramos = True
    else:        
#       if coste > umbral :
#            paramos = True
#       else:
#            paramos = False
#Se borra la linea de abajo
        paramos = False
            
        
    #Controlamos si hemos terminado la prueba
    if paramos == True:  #añadir umbral de potencia
        #Hemos acabado con los tramos de la prueba
        contador_resultados +=1
        #print("Registro %d , %d" %(contador_resultados, np.sum(registro_func)))
        
        with open("trace.txt", "a") as f: 
            f.write("Registro %d ,Potencia Media: %d , Tiempo Total %d \n" %(contador_resultados, potencia_media_func, tiempo_acum_func)) 
        i=0
        with open("trace_detail.txt", "a") as f_detail:
            for tramo in registro_func:
                i+=1
                f_detail.write("Registro %d ,Tramo %d :  %d watios \n" %(contador_resultados,i,tramo))
        
    else:
        #print("Entra en el tramo %d" %(lista_tramos.head(1)["tramo"].iloc[0]))
        #pwr = lista_tramos.head(1)["pwr"].iloc[0]
        porc = lista_tramos.head(1)["porc"].iloc[0]
        viento = lista_tramos.head(1)["viento_aparente"].iloc[0]
        #tiempo = lista_tramos.head(1)["time_seg"].iloc[0]
        
        for i in range(0,6):
            #velocidad=lista_tramos.head(1)["spd"].iloc[0]*(1+(i/10))
            potencia = potencia_entrada+(i*10)
            velocidad = model.predict([[potencia, porc, viento]])[0]
            
            nuevo_tiempo = lista_tramos.head(1)["dist"].iloc[0]/(velocidad/3.6)
            
            registro = registro_func.copy()
            registro.append(potencia)
            
            # Actualizamos valor del coste medio, pendiente
            potencia_acum = potencia_acum_func + (potencia * nuevo_tiempo)
            tiempo_acum = tiempo_acum_func + nuevo_tiempo
            potencia_media = potencia_acum / tiempo_acum
            
            
            simulador2(lista_tramos[:][1:], nivel+1,registro, potencia_entrada, potencia_media, tiempo_acum, potencia_acum)
        
        return None

In [56]:
import time
start_time = time.time()

nivel_incial = 0
registro = []

coste_medio = 0
tiempo_inicial = 0
potencia_media_incial = 0
potencia_entrada = 250

contador_resultados = 0

model = joblib.load('../Machine Learning/modelo_export.model') 

with open("trace.txt", "w") as f: 
            f.write("")
with open("trace_detail.txt", "w") as f_detail: 
            f_detail.write("")

        
print("Numero de combinaciones: %d" %(6**len(df)))       
#simulador(df, nivel_incial, registro, potencia_entrada, coste_medio)
simulador2(df, nivel_incial, registro, potencia_entrada, coste_medio, tiempo_inicial, potencia_media_incial)

print("--- %s seconds ---" % (time.time() - start_time))

Numero de combinaciones: 7776
--- 274.1446144580841 seconds ---


In [ ]:
contador_resultados